In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10b378ce0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10b379d60>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
class State(TypedDict):
      topic: str
      joke: str
      improved_joke: str
      final_joke: str

In [3]:
#nodes
def generate_joke(state:State):
      """first LLm call to genrate inital joke"""
      msg = llm.invoke(
          f"Generate a joke about a {state['topic']}",
      )
      return {"joke":msg.content}

def check_punchline(state:State):
      """Gate to check if joke is funny or not"""

      if "?" in state["joke"]:
          return "Fail"
      return "Pass"

In [4]:
def improve_joke(state:State):
      """Second LLM call to improve the joke"""
      msg = llm.invoke(
          f"Make this joke funnier: {state['joke']}",
      )
      return {"improved_joke":msg.content}

def polish_joke(state:State):
      """Final LLM call to polish the joke"""
      msg = llm.invoke(
          f"Polish this joke: {state['improved_joke']}",
      )
      return {"final_joke":msg.content}

In [7]:
workflow = StateGraph(State)

workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

workflow.add_edge(START, "generate_joke")
workflow.add_conditional_edges(
    "generate_joke", check_punchline,{
        "Fail": "improve_joke", "Pass": END
    }
)
workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("polish_joke", END)

graph =workflow.compile()


In [6]:
graph.invoke({"topic":"Generate a joke about a cat"})
# graph.visualize()

{'topic': 'Generate a joke about a cat',
 'joke': "Why don't cats play poker in the jungle? \n\nToo many cheetahs! 😹  \n",
 'improved_joke': "Here are a few ways to tweak the joke to make it funnier:\n\n**Playing with Puns:**\n\n*  Why don't cats play poker in the jungle? Too many cheetahs! They're always trying to **claw** their way to the top. 😹\n* Why don't cats play poker in the jungle? **Too many felines with a shady past!**  😹\n\n**Adding Absurdity:**\n\n* Why don't cats play poker in the jungle? **The stakes are too high!**  One wrong move and you could end up as a lion's lunch.😹\n* Why don't cats play poker in the jungle?  **The monkeys keep throwing bananas at the table!** 😹\n\n**Unexpected Twist:**\n\n* Why don't cats play poker in the jungle? **They prefer blackjack.**  😹 (This plays on the stereotype of cats liking to gamble.)\n\n**Remember:**  Humor is subjective! What one person finds funny, another might not.  Experiment with different variations and see what gets the be